In [1]:
!pip install langdetect
!pip install pyspellchecker

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     --- ----------------------------------- 81.9/981.5 kB 1.2 MB/s eta 0:00:01
     ------ ------------------------------- 174.1/981.5 kB 1.5 MB/s eta 0:00:01
     ----------- -------------------------- 307.2/981.5 kB 1.7 MB/s eta 0:00:01
     --------------- ---------------------- 409.6/981.5 kB 2.0 MB/s eta 0:00:01
     -------------------- ----------------- 532.5/981.5 kB 2.1 MB/s eta 0:00:01
     ------------------------ ------------- 624.6/981.5 kB 2.1 MB/s eta 0:00:01
     ---------------------------- --------- 747.5/981.5 kB 2.1 MB/s eta 0:00:01
     -------------------------------- ----- 839.7/981.5 kB 2.1 MB/s eta 0:00:01
     -------------------------------------  972.8/981.5 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing 


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB 1.4 MB/s eta 0:00:05
    --------------------------------------- 0.1/6.8 MB 2.1 MB/s eta 0:00:04
   - -------------------------------------- 0.3/6.8 MB 2.3 MB/s eta 0:00:03
   -- ------------------------------------- 0.4/6.8 MB 2.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/6.8 MB 2.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/6.8 MB 2.3 MB/s eta 0:00:03
   ---- ----------------------------------- 0.7/6.8 MB 2.3 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/6.8 MB 2.4 MB/s eta 0:00:03
   ----- ---------------------------------- 0.9/6.8 MB 2.3 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/6.8 MB 2.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.1/6.8 MB 2.3 MB/s eta 0:00:03
   ------- -------------------------------- 1.3/6.8 MB 2.4 MB/s eta 0:00:03
   -------- -------


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Import necessary libraries
import re

import pandas as pd
import numpy as np
import spacy
from spacy.language import Language
from spacy.tokens import Doc
from spacy.language import Language
from langdetect import detect, DetectorFactory, LangDetectException
from spacy.tokens import Doc
from spacy.language import Language

In [9]:
# Load and clean up reviews
df = pd.read_csv("processed_reviews.csv")

In [11]:
nlp = spacy.load('en_core_web_sm')
@Language.component("filter_length")
def filter_length(doc):
    # Count the number of word tokens in the document
    word_count = len([token for token in doc if token.is_alpha])

    # Check if the document has more than 1 word
    if word_count > 1:
        return doc
    else:
        # Return an empty Doc if the condition is not met
        return Doc(doc.vocab, words=[])

# Add the component to the pipeline
nlp.add_pipe("filter_length", name="length_filter", first=True)

<function __main__.filter_length(doc)>

In [12]:
@Language.component("filter_emojis")
def filter_emojis(doc):
    # Check if the document is empty (as a result of the previous filter)
    if len(doc) == 0:
        return doc

    # Regular expression pattern to match emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)

    # Remove emojis from the text
    new_text = emoji_pattern.sub(r'', doc.text)

    # Create a new Doc with the emoji-free text
    return Doc(doc.vocab, words=new_text.split())

# Add the component to the pipeline
nlp.add_pipe("filter_emojis", name="filter_emojis", after="filter_length")

ValueError: [E001] No component 'filter_length' found in pipeline. Available names: ['length_filter', 'tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner']

In [54]:
# Set seed for langdetect to get deterministic results
DetectorFactory.seed = 0

@Language.component("filter_language")
def filter_language(doc):
    # Check if the document is empty
    if len(doc) == 0:
        return doc

    try:
        # Detect the language of the text
        if detect(doc.text) == 'en':
            return doc
        else:
            # Return an empty Doc if the text is not English
            return Doc(doc.vocab, words=[])
    except LangDetectException:
        # Handle exception if language detection fails
        return Doc(doc.vocab, words=[])

# Add the component to the pipeline
nlp.add_pipe("filter_language", name="filter_language", after="filter_emojis")

<function __main__.filter_language(doc)>

In [55]:
# Custom component for keeping only standard English sentence symbols
@Language.component("filter_english_symbols")
def filter_english_symbols(doc):
    # Define a regular expression pattern to match all types of punctuation
    punctuation_pattern = re.compile(r'[^\w\s]+')

    # Use re.sub() to replace punctuation with an empty space in the entire text
    processed_text = re.sub(punctuation_pattern, ' ', doc.text)

    # Create a new Doc with the processed text
    new_doc = Doc(doc.vocab, words=processed_text.split())
    return new_doc

# Add the component to the pipeline
nlp.add_pipe("filter_english_symbols", name="filter_english_symbols", after="filter_emojis")

<function __main__.filter_english_symbols(doc)>

In [57]:
@Language.component("remove_stopwords")
def remove_stopwords(doc):
    # Check if the document is empty
    if len(doc) == 0:
        return doc

    # Remove stop words from the text
    stop_words = set(token.text for token in doc if token.is_stop and token.text.lower() not in {"not", "do", "can","don"})
    filtered_tokens = [token.text for token in doc if token.text not in stop_words]
    # Check if filtered tokens are empty
    if not filtered_tokens:
        return doc

    # Create a new Doc with the text without stop words
    new_doc = Doc(doc.vocab, words=filtered_tokens)
    return new_doc

# Add the component to the pipeline
nlp.add_pipe("remove_stopwords", name="remove_stopwords", after="filter_english_symbols")

<function __main__.remove_stopwords(doc)>

In [58]:
contractions = {
    "t": "not",
    "don": "do",
    "doesn": "does",
    "didn": "did",
    "won": "will",
    "cant": "cannot",
}

@Language.component("expand_contractions")
def expand_contractions(doc):
    # Check if the document is empty
    if len(doc) == 0:
        return doc

    # Expand contractions in the text
    expanded_tokens = [contractions.get(token.text, token.text) for token in doc]

    # Check if expanded tokens are empty
    if not expanded_tokens:
        return doc

    # Create a new Doc with the expanded text
    new_doc = Doc(doc.vocab, words=expanded_tokens)
    return new_doc

# Add the component to the pipeline
nlp.add_pipe("expand_contractions", name="expand_contractions", after="remove_stopwords")

<function __main__.expand_contractions(doc)>

In [59]:
@Language.component("preprocess_text")
def preprocess_text(doc):
    # Check if the document is empty
    if len(doc) == 0:
        return doc

    # Process the text: lowercase, remove punctuation, and lemmatize
    lemmatized_tokens = [token.lemma_.lower() for token in doc if token.is_alpha]

    if not lemmatized_tokens:
      return doc

    # Create a new Doc with the processed text
    # Note: This method preserves the original Doc's properties
    new_doc = Doc(doc.vocab, words=lemmatized_tokens)
    return new_doc

# Add the component to the pipeline
nlp.add_pipe("preprocess_text", name="preprocess_text", last=True)
# nlp.remove_pipe("text_preprocessor")

<function __main__.preprocess_text(doc)>

In [60]:
nlp.pipe_names
nlp.to_disk("preprocessing_service/pipeline")

In [49]:
parsed_reviews = []
for i in list(df.index):
  content = df.at[i, "content"]
  parsed = nlp(content)
  print("content:", content)
  print("parsed:", parsed)
  print("#"*10)
  parsed_reviews.append(parsed)

content: Gett van for no reason 😂😂😂
parsed: 
##########
content: better' than WhatsApp
parsed: well whatsapp 
##########
content: That was good app for me
parsed: good app 
##########
content: Love the app
parsed: love app 
##########
content: 🕳️🕳️🕳️
parsed: 
##########
content: Good app
parsed: good app 
##########
content: Nice app
parsed: nice app 
##########
content: telegram mare call par nai ban rhi
parsed: 
##########
content: now how to mute stories?
parsed: mute story 
##########
content: Good parfom sab 1
parsed: 
##########
content: Nice
parsed: 
##########
content: Nise
parsed: 
##########
content: VERY GOOD
parsed: 
##########
content: Good
parsed: 
##########
content: When I enter my phone number it show I am banned I email support telegram but I got no answer😭
parsed: enter phone number ban email support telegram get answer 
##########
content: Barrs users login for no tangible reason, null response on login issues emailing, nice UX/UI, but functions,..void/minimal will 

In [22]:
df["parsedContent"] = parsed_reviews

In [23]:
# Parsing reviews into usable pieces of text leaves us with half of the reviews
print(df.shape[0])
df_use = df.loc[df["parsedContent"].apply(lambda x: len(x)>0)].copy()
print(df_use.shape[0])

1200
593


In [24]:
# Now we can try to create training data and fit the model
df_use["sentiment_pos"] = df_use["score"].apply(lambda x: x>3)
df_use['cats'] = [{'POSITIVE': y==True, 'NEGATIVE': y==False} for y in df_use['sentiment_pos']]
df_use[["content", "sentiment_pos"]].head()

,content,sentiment_pos
1,better' than WhatsApp,True
2,That was good app for me,True
3,Love the app,True
5,Good app,True
6,Nice app,True


In [25]:
# Randomly sample for a training dataset, use the rest for testing
df_train = df_use.sample(frac=0.7, random_state=42).copy()
df_dev = df_use.drop(df_train.index, inplace=False).sample(frac=0.5, random_state=42).copy()
df_test = df_use.drop(df_train.index).drop(df_dev.index).copy()

In [28]:
df_train["parsedContent"].head()

60     (don, not, do, business, telegram, people, do,...
139                                 (easy, install, pls)
490    (long, message, app, turn, social, medium, app...
916                       (app, need, heart, get, break)
393    (suck, app, keep, loading, appss, facebook, wo...
Name: parsedContent, dtype: object

In [29]:
from spacy.training import Example
from spacy.tokens import DocBin

# Create training, validation and test sets and save them in .spacy format
bin = DocBin()

for i in list(df_train.index):
  example = Example.from_dict(df_train.at[i, "parsedContent"], {'cats': df_train.at[i, 'cats']})
  bin.add(example.reference)

bin.to_disk('train.spacy')

bin = DocBin()

for i in list(df_dev.index):
  example = Example.from_dict(df_dev.at[i, "parsedContent"], {'cats': df_dev.at[i, 'cats']})
  bin.add(example.reference)

bin.to_disk('dev.spacy')

bin = DocBin()

for i in list(df_test.index):
  example = Example.from_dict(df_test.at[i, "parsedContent"], {'cats': df_test.at[i, 'cats']})
  bin.add(example.reference)

bin.to_disk('test.spacy')

In [30]:
# create a configuration file with default settings
!python -m spacy init config -p textcat config.cfg

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: textcat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [31]:
# train the model
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       73.68    0.74
  7     200         32.95       82.05    0.82
 15     400         17.95       80.96    0.81
 26     600         12.09       83.52    0.84
 39     800          9.03       83.52    0.84
 55    1000          7.05       83.52    0.84
 75    1200          5.76       80.26    0.80
 99    1400          4.79       80.26    0.80
128    1600          4.15       80.26    0.80
164    1800          3.64       80.26    0.80
207    2000          3.26       80.26    0.80
257    2200          3.09       79.18    0.79
✔ Saved pipeline to output directory
ou

In [32]:
# test the saved model
nlp = spacy.load("output/model-best")

In [54]:
# evaluate on the first 5 reviews
for i in list(df_test.index)[:5]:
  content = df_test.at[i, "parsedContent"]
  positive_sentiment = df_test.at[i, "sentiment_pos"]
  print("First 5 words:", content[:5], "predicted positive probability:", round(nlp(content).cats["POSITIVE"], 2), "true positive sentiment:", positive_sentiment)

First 5 words: nice app predicted positive probability: 0.86 true positive sentiment: True
First 5 words: people predicted positive probability: 0.58 true positive sentiment: True
First 5 words: app apps good predicted positive probability: 0.89 true positive sentiment: True
First 5 words: unfortunately female pikachu not want predicted positive probability: 0.12 true positive sentiment: False
First 5 words: love feature excellent app want predicted positive probability: 0.93 true positive sentiment: True


In [3]:
import requests, json
data = {
    "text_to_process": "better' than WhatsApp"
}


response = requests.post('http://localhost:5000/process_and_predict', json = data)
print(response.text)

{"positive_probability":0.714198112487793,"processed_text":"well whatsapp "}

